In [1]:
import sys
sys.path.insert(0,'..')
from IPython.display import display, HTML
if not sys.warnoptions:
    import warnings
# import imblearn
from collections import Counter
from sklearn.preprocessing import LabelEncoder
# from imblearn.over_sampling import SMOTE
from matplotlib.ticker import PercentFormatter
from os import listdir
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import SkyCoord
import astropy.constants as const
from astropy.cosmology import WMAP9 as cosmo
from datetime import datetime
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
from matplotlib.cm import register_cmap
# from matplotlib_venn import venn3

from numpy.lib.stride_tricks import as_strided
import pickle
from scipy import stats
from datetime import datetime
# from spectral_cube import SpectralCube
# from imblearn.under_sampling import RandomUnderSampler

In [2]:
freq_dict = {"mask_1245mosB.fits": [1401252439.970,1425092772.001],
"mask_1245mosC.fits": [1381037599.970,1404877932.001],
"mask_1245mosD.fits": [1360822751.970,1384663084.001],
"mask_1245mosE.fits": [1340607911.970,1364448244.001],
"mask_1245mosF.fits": [1320393067.970,1344233400.001],
"mask_1245mosG.fits": [1300178221.970,1324018554.001],
"mask_1245mosH.fits": [1279963378.910,1303803710.941],
"mask_1353mosB.fits": [1401179199.970,1425019532.001],
"mask_1353mosC.fits": [1380964351.970,1404804684.001],
"mask_1353mosD.fits": [1360749511.970,1384589844.001],

"mask_1353mosE.fits": [1340534667.970,1364375000.001],
"mask_1353mosF.fits": [1320319823.970,1344160156.001],
"mask_1353mosG.fits": [1300104979.970,1323945312.001],
"mask_1353mosH.fits": [1279890136.720,1303730468.751]}
freq_df = pd.DataFrame(freq_dict.values(), columns=["freq_1", "freq_2"])
freq_df["file_names"] = freq_dict.keys()
freq_df["freq_1_round"] = np.round(freq_df['freq_1']*1e-6).astype(int).astype(str)
freq_df["freq_2_round"] = np.round(freq_df['freq_2']*1e-6).astype(int).astype(str)
freq_df['freq_labels'] = freq_df[['freq_1_round', 'freq_2_round']].agg('-'.join, axis=1)
freq_df.columns = freq_df.columns.map(''.join)
freq_df['mos_name'] = freq_df.file_names.str.replace("mask_", "").str.replace(".fits", "")

In [4]:
real_cat_df = pd.DataFrame()
for file in listdir("../../results/catalogs/ground_truth"):
    if "properties"in file:
        temp_df = pd.read_csv(f"../../results/catalogs/ground_truth/{file}", index_col=0)
        temp_df["mos_name"] = file.split("_")[0]
        temp_df = temp_df.reset_index()
        real_cat_df = real_cat_df.append(temp_df)
real_cat_df = real_cat_df.reset_index(drop=True)
real_cat_df["z_pos"] = ((real_cat_df["bbox-3"]+ real_cat_df["bbox-0"])/2).astype(int)
real_cat_df["x_pos"] = ((real_cat_df["bbox-4"]+ real_cat_df["bbox-1"])/2).astype(int)
real_cat_df["y_pos"] = ((real_cat_df["bbox-5"]+ real_cat_df["bbox-2"])/2).astype(int)
real_cat_df[real_cat_df.type == "real"].to_csv("../../results/catalogs/REAL_catalog.txt")

In [5]:
mos_names = [i.split("_")[0] for i in listdir("../../results/catalogs/ground_truth") if "inserted" in i]
mock_galaxy_gt_df = pd.DataFrame()
k = 0
for mos_name in mos_names:
    temp_df1 = pd.read_csv(f"../../results/catalogs/ground_truth/{mos_name}_properties.csv")
    temp_df2 = pd.read_csv(f"../../results/catalogs/ground_truth/{mos_name}_inserted.csv")
    temp_df1["z_pos"] = ((temp_df1["bbox-3"]+ temp_df1["bbox-0"])/2).astype(int)
    temp_df1["x_pos"] = ((temp_df1["bbox-4"]+ temp_df1["bbox-1"])/2).astype(int)
    temp_df1["y_pos"] = ((temp_df1["bbox-5"]+ temp_df1["bbox-2"])/2).astype(int)
    overlap_df = pd.merge(temp_df1[temp_df1.type=='mock'], temp_df2, on=["z_pos", "x_pos", "y_pos"], how='outer', indicator=True)
    left_only = overlap_df[overlap_df._merge == 'left_only']
    right_only = overlap_df[overlap_df._merge == 'right_only']
    both = overlap_df[overlap_df._merge == 'both'][overlap_df.columns[:-1]]
    nearest_overlap_df = pd.DataFrame()
    for i, row in right_only.iterrows():
        distances = np.sqrt((left_only.z_pos-row.z_pos)**2 + (left_only.x_pos-row.x_pos)**2 + (left_only.y_pos-row.y_pos)**2)
        mapped_row = left_only[distances == distances.min()][right_only.columns[:-8]]
        joined_df = mapped_row.reset_index(drop=True).join(pd.DataFrame([row]).reset_index(drop=True)[["z_pos", "x_pos", "y_pos", "gal_file", "orig_mass", "new_mass", "mos_name"]])
        joined_df["overlap"] = distances.min()
        nearest_overlap_df = nearest_overlap_df.append(joined_df)
    # Take closest if 2
    nearest_overlap_df.loc[nearest_overlap_df.groupby('label')['overlap'].transform('min').eq(nearest_overlap_df['overlap'])].reset_index(drop=True)
    nearest_overlap_df = nearest_overlap_df[nearest_overlap_df.columns[:-1]]
    all_mapped_df = nearest_overlap_df.append(both).reset_index(drop=True)
    all_mapped_df["mos_name"] == mos_name
    mock_galaxy_gt_df = mock_galaxy_gt_df.append(all_mapped_df)
mock_galaxy_gt_df.to_csv("../../results/catalogs/MOCK_catalog.txt")

In [87]:
mock_cat_df = pd.read_csv("../../results/catalogs/MOCK_catalog.txt", index_col=0)
real_cat_df = pd.read_csv("../../results/catalogs/REAL_catalog.txt", index_col=0)
# mock_cat_df.append(real_cat_df).to_csv("../../results/catalogs/MASK_catalog.txt")

In [92]:
mto_cat_df = pd.read_csv("../../results/catalogs/MTO_catalog.txt", index_col=0)
vnet_cat_df = pd.read_csv("../../results/catalogs/VNET_catalog.txt", index_col=0)
sofia_cat_df = pd.read_csv("../../results/catalogs/SOFIA_catalog.txt", index_col=0)
mask_cat_df = pd.read_csv("../../results/catalogs/MASK_catalog.txt", index_col=0)

In [93]:
mto_cat_df["log10mass"] = np.log10(mto_cat_df.hi_mass)
vnet_cat_df["log10mass"] = np.log10(vnet_cat_df.hi_mass)
sofia_cat_df["log10mass"] = np.log10(sofia_cat_df.hi_mass)
# mask_cat_df["log10mass"] = np.log10(mass_cat_df.new_mass)
mto_cat_df["max_loc"] = mto_cat_df.max_loc.apply(literal_eval)
vnet_cat_df["max_loc"] = vnet_cat_df.max_loc.apply(literal_eval)
sofia_cat_df["max_loc"] = sofia_cat_df.max_loc.apply(literal_eval)
mask_cat_df["max_loc"] = mask_cat_df.max_loc.apply(literal_eval)
mto_cat_df["max_loc_dist"] = mto_cat_df.max_loc.apply(lambda x: np.sqrt(x[0]**2 + x[1]**2 + x[2]**2))
vnet_cat_df["max_loc_dist"] = vnet_cat_df.max_loc.apply(lambda x: np.sqrt(x[0]**2 + x[1]**2 + x[2]**2))
sofia_cat_df["max_loc_dist"] = sofia_cat_df.max_loc.apply(lambda x: np.sqrt(x[0]**2 + x[1]**2 + x[2]**2))
mask_cat_df["max_loc_dist"] = mask_cat_df.max_loc.apply(lambda x: np.sqrt(x[0]**2 + x[1]**2 + x[2]**2))

Real Galaxies:
- V-Net is correct
- SoFiA is +- 1/2 difference
- MTO is +- 10 over

Mock Galaxies:
- All way under - not matching properly
- try with area_gt and nearest max_loc

In [99]:
mos_names = [i.split("_")[0] for i in listdir("../../results/catalogs/ground_truth") if "inserted" in i]
def connect_detections_to_ground_truth(detection_catalogue, gt_catalgue, mos_names, types):
    mapped_df = pd.DataFrame()
    k = 0
    for mos_name in mos_names:
        overlap_df = pd.merge_asof(detection_catalogue[(detection_catalogue.type==types) & (detection_catalogue.mos_name == mos_name)].sort_values('max_loc_dist'), gt_catalgue[(gt_catalgue.type==types) & (gt_catalgue.mos_name == mos_name)].sort_values('max_loc_dist'), on="max_loc_dist", suffixes=("_detection", "_mask"), direction="nearest")
        mapped_df = mapped_df.append(overlap_df)
    return mapped_df


In [112]:
mto_mock_df = connect_detections_to_ground_truth(detection_catalogue=mto_cat_df, gt_catalgue=mask_cat_df, mos_names=mos_names, types="mock")
vnet_mock_df = connect_detections_to_ground_truth(detection_catalogue=vnet_cat_df, gt_catalgue=mask_cat_df, mos_names=mos_names, types="mock")
sofia_mock_df = connect_detections_to_ground_truth(detection_catalogue=sofia_cat_df, gt_catalgue=mask_cat_df, mos_names=mos_names, types="mock")

mto_real_df = connect_detections_to_ground_truth(detection_catalogue=mto_cat_df, gt_catalgue=mask_cat_df, mos_names=mos_names, types="real")
vnet_real_df = connect_detections_to_ground_truth(detection_catalogue=vnet_cat_df, gt_catalgue=mask_cat_df, mos_names=mos_names, types="real")
sofia_real_df = connect_detections_to_ground_truth(detection_catalogue=sofia_cat_df, gt_catalgue=mask_cat_df, mos_names=mos_names, types="real")

In [113]:
len(mto_cat_df[(mto_cat_df.type == "mock")]), len(mto_mock_df), len(mto_cat_df[(mto_cat_df.type == "real")]), len(mto_real_df)

(618, 618, 68, 68)

In [115]:
len(vnet_cat_df[(vnet_cat_df.type == "mock")]), len(vnet_mock_df), len(vnet_cat_df[(vnet_cat_df.type == "real")]), len(vnet_real_df)

(1707, 1707, 137, 137)

In [117]:
len(sofia_cat_df[(sofia_cat_df.type == "mock")]), len(sofia_mock_df), len(sofia_cat_df[(sofia_cat_df.type == "real")]), len(sofia_real_df)

(1228, 1228, 194, 194)